## Summary

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import graphviz

## Import Datasets

In [2]:
enron_df = pd.read_csv('datasets/explored/enron-emails-explored.csv')
mal_df = pd.read_csv('datasets/explored/malicious-emails-explored.csv')

Create the Training sets

In [3]:
import random


part_1 = enron_df.sample(11871, ignore_index=True, random_state=1)
part_2 = mal_df
rand = round(random.random())

training_set = (pd.concat([part_1, part_2], ignore_index=True)).sample(frac=1, ignore_index=True, random_state=rand)

In [4]:
training_set.columns

Index(['Unnamed: 0', 'body', 'subject', 'raw_mail', 'from', 'to', 'status',
       'date', 'parsed_from', 'parsed_to', 'parsed_date', 'malicious',
       'content_type', 'charset', 'content_transfer_encoding', 'from_domain',
       'to_domain', 'html', 'javascript', 'css', 'html_form', 'html_iframe',
       'URLs_in_message', 'subject_len', 'subject_entropy', 'body_entropy',
       'attachement'],
      dtype='object')

In [5]:
features = ['html', 
            'javascript', 
            'css', 
            'html_form', 
            'html_iframe',
            'URLs_in_message', 
            'subject_len', 
            'subject_entropy', 
            'body_entropy',
            'attachement']

In [6]:
X = training_set[features]
y = training_set.malicious

train_X, val_X, train_y, val_y = train_test_split(X, y,random_state = rand)

In [7]:
X

,html,javascript,css,html_form,html_iframe,URLs_in_message,subject_len,subject_entropy,body_entropy,attachement
0,False,False,False,False,False,0,20,3.684184,3.605338,False
1,False,False,False,False,False,2,62,4.068553,4.855729,False
2,False,False,False,False,False,0,21,3.725651,4.587015,False
3,True,False,False,True,False,3,28,4.039149,4.745707,False
4,False,False,False,False,False,0,38,4.145972,4.562487,False
...,...,...,...,...,...,...,...,...,...,...
19995,False,False,False,False,False,0,13,3.546594,4.987845,False
19996,True,False,True,False,False,10,26,3.715454,5.191878,False
19997,False,False,False,False,False,0,14,3.182006,4.993333,False
19998,False,False,False,False,False,0,30,3.789898,4.367640,False


## Decision Tree

In [8]:
dts_clf = DecisionTreeClassifier(random_state=rand)
dts_model = dts_clf.fit(train_X, train_y)
pred_y = dts_model.predict(val_X)

print(f"Number of mislabeled points out of a total: {len(val_X)} points: {(val_y != pred_y).sum()}")
print(f"The accuracy of the model is: {((val_y == pred_y).sum() / len(val_X)) * 100}%")

Number of mislabeled points out of a total: 5000 points: 824
The accuracy of the model is: 83.52000000000001%


## KNN

In [10]:
knn_clf = KNeighborsClassifier()
knn_model = knn_clf.fit(train_X, train_y)
pred_y = knn_model.predict(val_X)

print(f"Number of mislabeled points out of a total: {len(val_X)} points: {(val_y != pred_y).sum()}")
print(f"The accuracy of the model is: {((val_y == pred_y).sum() / len(val_X)) * 100}%")

Number of mislabeled points out of a total: 5000 points: 904
The accuracy of the model is: 81.92%


## Random Forest

In [11]:
rf_clf = RandomForestClassifier(random_state=rand)
rf_model = rf_clf.fit(train_X, train_y)
pred_y = rf_model.predict(val_X)

print(f"Number of mislabeled points out of a total: {len(val_X)} points: {(val_y != pred_y).sum()}")
print(f"The accuracy of the model is: {((val_y == pred_y).sum() / len(val_X)) * 100}%")

Number of mislabeled points out of a total: 5000 points: 646
The accuracy of the model is: 87.08%


## Naive Bayes

In [12]:
nb_clf = GaussianNB()
nb_model = nb_clf.fit(train_X, train_y)
pred_y = nb_model.predict(val_X)

print(f"Number of mislabeled points out of a total: {len(val_X)} points: {(val_y != pred_y).sum()}")
print(f"The accuracy of the model is: {((val_y == pred_y).sum() / len(val_X)) * 100}%")

Number of mislabeled points out of a total: 5000 points: 1336
The accuracy of the model is: 73.28%
